# Download data from warehouse -> add it to our private GCS bucket as a parquet -> clean it -> add it to the public GCS bucket

In [7]:
from datetime import datetime
from functools import cache
from pathlib import Path

import geopandas as gpd
import pandas as pd
from calitp_data_analysis.gcs_geopandas import GCSGeoPandas
from calitp_data_analysis.gcs_pandas import GCSPandas

In [8]:
from shared_utils import (
    bq_utils,
    geo_utils,
    gtfs_utils_v2,
    portfolio_utils,
    publish_utils,
    rt_dates,
)

In [9]:
from shared_utils import geo_utils, gtfs_utils_v2, portfolio_utils, publish_utils
from update_vars import (
    GTFS_DATA_DICT,
    analysis_month,
    file_name,
    last_year,
    previous_month,
)

In [10]:
from calitp_data_analysis import geography_utils, utils

In [11]:
import google.auth
import pandas_gbq

credentials, project = google.auth.default()

In [12]:
from calitp_data_analysis.sql import get_engine

# from calitp_data_analysis.tables import tbls

db_engine = get_engine()

In [13]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [14]:
PUBLIC_GCS = GTFS_DATA_DICT.gcs_paths.PUBLIC_GCS

In [15]:
@cache
def gcs_geopandas():
    return GCSGeoPandas()

In [16]:
@cache
def gcs_pandas():
    return GCSPandas()

In [17]:
analysis_month

'2025-12-01'

In [18]:
file_name

'2025_12'

In [19]:
PROD_PROJECT = "cal-itp-data-infra"
STG_PROJECT = "cal-itp-data-infra-staging"
STG_MART = "tiffany_mart_gtfs_rollup"
MONTH_DATE_COL = "month_first_day"

In [20]:
TRANSIT_MART = "mart_transit_database"

In [21]:
PROD_MART = "mart_gtfs_rollup"

## 1) Crosswalk

In [ ]:
def load_crosswalk() -> pd.DataFrame:
    df = bq_utils.download_table(
        project_name="cal-itp-data-infra",
        dataset_name="mart_transit_database",
        table_name="bridge_gtfs_analysis_name_x_ntd",
        date_col=None,
    )
    df2 = (
        df.dropna(subset=["ntd_id", "ntd_id_2022"])
        .drop_duplicates(
            subset=["analysis_name", "organization_name", "schedule_gtfs_dataset_name"]
        )
        .reset_index()
    )

    df2 = df2.rename(columns={"schedule_gtfs_dataset_name": "name"})

    df2["caltrans_district_int"] = df2.caltrans_district
    df2.caltrans_district = df2.caltrans_district.apply(lambda x: '{0:0>2}'.format(x)) 
    
    df2["caltrans_district"] = (
        df2.caltrans_district.astype(str) + "-" + df2.caltrans_district_name
    )

    df2 = df2[
        [
            "name",
            "analysis_name",
            "county_name",
            "caltrans_district",
            "caltrans_district_int",
            "ntd_id",
            "ntd_id_2022",
        ]
    ]

    gcs_pandas().data_frame_to_parquet(df2, f"{GTFS_DATA_DICT.gcs_paths.DIGEST_GCS}processed/{GTFS_DATA_DICT.gtfs_digest_rollup.crosswalk}_{file_name}.parquet")
    
    return df2

In [ ]:
crosswalk_df = load_crosswalk()

In [33]:
crosswalk_df = bq_utils.download_table(
        project_name="cal-itp-data-infra",
        dataset_name="mart_transit_database",
        table_name="bridge_gtfs_analysis_name_x_ntd",
        date_col=None,
    )

Downloading: 100%|██████████|
query: SELECT * FROM  `cal-itp-data-infra`.`mart_transit_database`.`bridge_gtfs_analysis_name_x_ntd`


In [38]:
crosswalk_df.columns

Index(['organization_name', 'organization_source_record_id',
       'schedule_source_record_id', 'schedule_gtfs_dataset_name',
       'analysis_name', 'regional_feed_type', 'county_name',
       'caltrans_district', 'caltrans_district_name', 'caltrans_district_full',
       'ntd_id', 'ntd_id_2022', 'rtpa_name', 'mpo_name'],
      dtype='object')

### YML

In [ ]:
# yml_df = crosswalk_df[["caltrans_district", "caltrans_district_name", "analysis_name"]]

## 2) NTD Data

In [ ]:
ntd_query_sql = f"""
        SELECT 
        number_of_state_counties,
        primary_uza_name,
        density,
        number_of_counties_with_service,
        state_admin_funds_expended,
        service_area_sq_miles,
        population,
        service_area_pop,
        subrecipient_type,
        primary_uza_code,
        reporter_type,
        organization_type,
        agency_name,
        voms_pt,
        voms_do,
        ntd_id,
        year
        FROM `cal-itp-data-infra-staging`.`mart_ntd`.`dim_annual_agency_information`
        WHERE state = 'CA' AND _is_current = TRUE
    """


mobility_query_sql = f"""
            SELECT
            agency_name,
            counties_served,
            hq_county,
            is_public_entity,
            is_publicly_operating,
            funding_sources,
            on_demand_vehicles_at_max_service,
            vehicles_at_max_service
            FROM
            cal-itp-data-infra.mart_transit_database.dim_mobility_mart_providers  
            """


def load_mobility(query: str) -> pd.DataFrame:
    with db_engine.connect() as connection:
        df = pd.read_sql(query, connection)
    df2 = df.sort_values(
        by=["on_demand_vehicles_at_max_service", "vehicles_at_max_service"],
        ascending=[False, False],
    )
    df3 = df2.groupby("agency_name").first().reset_index()
    return df3


def load_ntd(query: str) -> pd.DataFrame:
    with db_engine.connect() as connection:
        df = pd.read_sql(query, connection)
    df2 = df.sort_values(by=df.columns.tolist(), na_position="last")
    df3 = df2.groupby("agency_name").first().reset_index()
    return df3

In [ ]:
def merge_ntd_mobility(ntd_query: str, mobility_query: str) -> pd.DataFrame:
    """
    Merge NTD (dim_annual_ntd_agency_information) with
    mobility providers (dim_mobility_mart_providers)
    and dedupe and keep 1 row per agency.
    """
    ntd = load_ntd(ntd_query)
    mobility = load_mobility(mobility_query)
    crosswalk = load_crosswalk()[["analysis_name", "ntd_id_2022"]]

    m1 = pd.merge(mobility, ntd, how="inner", on="agency_name")

    m1 = m1.drop_duplicates(subset="agency_name").reset_index(drop=True)

    # Wherever possible, allow nullable integers. These columns are integers, but can be
    # missing if we don't find corresponding NTD info
    integrify_cols = [
        "number_of_state_counties",
        "number_of_counties_with_service",
        "service_area_sq_miles",
        "service_area_pop",
        "on_demand_vehicles_at_max_service",
        "vehicles_at_max_service",
        "voms_pt",
        "voms_do",
        "year",
    ]
    m1[integrify_cols] = m1[integrify_cols].astype("Int64")

    # Merge with crosswalk to get analysis_name
    m1 = pd.merge(
        m1, crosswalk, left_on=["ntd_id"], right_on=["ntd_id_2022"], how="inner"
    )

    # m1.columns = m1.columns.str.replace("_", " ").str.title()
    gcs_pandas().data_frame_to_parquet(m1,  f"{GTFS_DATA_DICT.gcs_paths.DIGEST_GCS}processed/{GTFS_DATA_DICT.gtfs_digest_rollup.ntd_profile}_{file_name}.parquet")

    return m1

In [ ]:
ntd_data_df = merge_ntd_mobility(
    ntd_query=ntd_query_sql, mobility_query=mobility_query_sql
)

In [ ]:
len(ntd_data_df)

## 3) schedule_rt_route_direction_summary
* Add Caltrans District to organize 

In [ ]:
def load_schedule_rt_route_direction_summary(
    project_name: str,
    date_col: str,
    dataset_name: str,
    start_date: str,
    end_date: str,
    file_name: str,
) -> pd.DataFrame:
    df = bq_utils.download_table(
        project_name=project_name,
        dataset_name=dataset_name,
        table_name=GTFS_DATA_DICT.gtfs_digest_rollup.schedule_rt_route_direction,
        date_col=date_col,
        start_date=start_date,
        end_date=end_date,
    )

    # Merge with crosswalk
    crosswalk_df = load_crosswalk()[["name", "analysis_name"]]
    m1 = pd.merge(df, crosswalk_df, on=["name"], how="inner")

    gcs_pandas().data_frame_to_parquet(m1,  f"{GTFS_DATA_DICT.gcs_paths.DIGEST_GCS}raw/{GTFS_DATA_DICT.gtfs_digest_rollup.schedule_rt_route_direction}_{file_name}.parquet")
    
    return m1

In [ ]:
schedule_rt_route_direction_summary = load_schedule_rt_route_direction_summary(
    project_name=PROD_PROJECT,
    date_col=MONTH_DATE_COL,
    dataset_name=PROD_MART,
    start_date=last_year,
    end_date=analysis_month,
    file_name=file_name,
)

In [ ]:
schedule_rt_route_direction_summary.sample()

In [ ]:
schedule_rt_route_direction_summary.loc[
    schedule_rt_route_direction_summary.name.str.contains("511 Muni")
].shape

In [ ]:
crosswalk_df = load_crosswalk()[["name", "analysis_name"]]

In [ ]:
crosswalk_df.loc[crosswalk_df.analysis_name.str.contains("San Francisco")]

In [ ]:
def prep_schedule_rt_route_direction_summary(file_name: str) -> pd.DataFrame:

    df = gcs_pandas().read_parquet(f"{GTFS_DATA_DICT.gcs_paths.DIGEST_GCS}raw/{GTFS_DATA_DICT.gtfs_digest_rollup.schedule_rt_route_direction}_{file_name}.parquet")

    # Prepare data for portfolio
    df2 = df[
        [
            "month_first_day",
            "analysis_name",
            "route_name",
            "direction_id",
            "frequency_all_day",
            "frequency_offpeak",
            "frequency_peak",
            "daily_service_hours",
            "daily_trips_peak",
            "daily_trips_offpeak",
            "daily_trips_all_day",
            "day_type",
            "route_type",
            "route_typology",
        ]
    ]

    # Drop duplicates
    df2 = df2.drop_duplicates().reset_index()

    # Clean columns
    df2.route_typology = df2.route_typology.str.title()
    df2.columns = df2.columns.str.replace("_", " ").str.title()
    df2["Month First Day"] = pd.to_datetime(df2["Month First Day"]).dt.strftime("%m/%Y")
    df2 = df2.rename(
        columns={
            "Direction Id": "Direction",
            "Month First Day": "Date",
            "Route Name": "Route",
        }
    )

    # Add some new columns
    df2["Daily Service Minutes"] = df2["Daily Service Hours"] * 60
    df2["Average Scheduled Minutes"] = (
        df2["Daily Service Minutes"] / df2["Daily Trips All Day"]
    )
    df2["Average Scheduled Minutes"].describe()
    df2["Headway All Day"] = 60 / df2["Frequency All Day"]
    df2["Headway Peak"] = 60 / df2["Frequency Peak"]
    df2["Headway Offpeak"] = 60 / df2["Frequency Offpeak"]

    # Save
    gcs_pandas().data_frame_to_parquet(df2, f"{GTFS_DATA_DICT.gcs_paths.DIGEST_GCS}processed/{GTFS_DATA_DICT.gtfs_digest_rollup.schedule_rt_route_direction}_{file_name}.parquet")
    
    return df2

In [ ]:
schedule_rt_route_direction_summary_df = prep_schedule_rt_route_direction_summary(
    file_name=file_name,
)

In [ ]:
schedule_rt_route_direction_summary_df.sample()

In [ ]:
sf = schedule_rt_route_direction_summary_df.loc[
    schedule_rt_route_direction_summary_df["Analysis Name"].str.contains(
        "City and County of San Francisco"
    )
]

In [ ]:
sf.shape

## 4) fct_monthly_operator_summary
* Add Caltrans District + Legislative District
* Missing many D3 operators: Roseville, EL Dorado, Glenn, Placer, Sac, Thaoe, Truckee, Yuba Sutter

In [29]:
def load_operator_summary(
    project_name: str,
    date_col: str,
    dataset_name: str,
    start_date: str,
    end_date: str,
    file_name: str,
) -> pd.DataFrame:
    df = bq_utils.download_table(
        project_name=project_name,
        dataset_name=dataset_name,
        table_name=GTFS_DATA_DICT.gtfs_digest_rollup.operator_summary,
        date_col=date_col,
        start_date=start_date,
        end_date=end_date,
    )

    crosswalk_url = f"{GTFS_DATA_DICT.gcs_paths.DIGEST_GCS}processed/{GTFS_DATA_DICT.gtfs_digest_rollup.crosswalk}_{file_name}.parquet"

    crosswalk_df = gcs_pandas().read_parquet(crosswalk_url)[["analysis_name", "caltrans_district"]]

    """
    m1 = pd.merge(df, crosswalk_df, on="analysis_name", how="inner").drop_duplicates().reset_index()

    gcs_pandas().data_frame_to_parquet(
        m1,
        f"{GTFS_DATA_DICT.gcs_paths.DIGEST_GCS}raw/"
        f"{GTFS_DATA_DICT.gtfs_digest_rollup.operator_summary}_{file_name}.parquet"
    )
    """
    return df

In [30]:
monthly_operator_summary_df = load_operator_summary(
    project_name=PROD_PROJECT,
    date_col=MONTH_DATE_COL,
    dataset_name=PROD_MART,
    start_date=last_year,
    end_date=analysis_month,
    file_name=file_name,
)

Downloading: 100%|██████████|
query: SELECT * FROM  `cal-itp-data-infra`.`mart_gtfs_rollup`.`fct_monthly_operator_summary` WHERE month_first_day >= DATE('2024-12-01') AND month_first_day <= DATE('2025-12-01')


In [35]:
PROD_PROJECT

'cal-itp-data-infra'

In [36]:
PROD_MART

'mart_gtfs_rollup'

In [37]:
GTFS_DATA_DICT.gtfs_digest_rollup.operator_summary

'fct_monthly_operator_summary'

In [32]:
monthly_operator_summary_df.columns

Index(['month', 'year', 'month_first_day', 'schedule_name',
       'schedule_base64_url', 'vp_name', 'vp_base64_url', 'tu_name',
       'tu_base64_url', 'day_type', 'n_trips', 'daily_trips',
       'ttl_service_hours', 'n_routes', 'n_shapes', 'n_stops', 'n_days',
       'vp_messages_per_minute', 'n_vp_trips', 'daily_vp_trips',
       'pct_vp_trips', 'pct_vp_service_hours', 'tu_messages_per_minute',
       'n_tu_trips', 'daily_tu_trips', 'pct_tu_trips', 'pct_tu_service_hours'],
      dtype='object')

In [ ]:
monthly_operator_summary_df = monthly_operator_summary_df.fillna("NA")

In [ ]:
def find_ops_monthly_summary(operator:str):
    
    display(monthly_operator_summary_df.loc[monthly_operator_summary_df.schedule_name.str.contains(operator)].shape)
    display(monthly_operator_summary_df.loc[monthly_operator_summary_df.vp_name.str.contains(operator)].shape)
    
    display(monthly_operator_summary_df.loc[monthly_operator_summary_df.tu_name.str.contains(operator)].shape)
    display(monthly_operator_summary_df.loc[monthly_operator_summary_df.analysis_name.str.contains(operator)].shape)

In [ ]:
# crosswalk_df.loc[crosswalk_df.caltrans_district.str.contains("11")]

In [ ]:
find_ops_monthly_summary("Imperial")

In [ ]:
find_ops_monthly_summary("North County")

In [ ]:
find_ops_monthly_summary("Yuma County")

In [ ]:
crosswalk_df = load_crosswalk()[["analysis_name", "caltrans_district"]]

In [ ]:
crosswalk_df.loc[crosswalk_df.caltrans_district.str.contains("12")]

In [ ]:
find_ops_monthly_summary("Laguna")

In [ ]:
find_ops_monthly_summary("Anaheim Transportation Network")

In [ ]:
find_ops_monthly_summary("Orange County Transportation Authority")

In [ ]:
crosswalk_df.loc[crosswalk_df.caltrans_district.str.contains("3")]

In [ ]:
find_ops_monthly_summary("Roseville")

In [ ]:
monthly_operator_summary_df[["schedule_name"]].drop_duplicates().sort_values(by = ["schedule_name"])

In [ ]:
crosswalk_df = load_crosswalk()[["analysis_name", "caltrans_district"]]

In [ ]:
crosswalk_df.loc[crosswalk_df.caltrans_district.str.contains("Marysville")][["analysis_name"]].drop_duplicates()

In [ ]:
monthly_operator_summary_df.loc[monthly_operator_summary_df.caltrans_district.str.contains("Marysville")][["analysis_name"]].drop_duplicates()

In [ ]:
len(monthly_operator_summary_df.drop_duplicates())

In [ ]:
def prep_operator_summary(file_name: str) -> pd.DataFrame:

    df = gcs_pandas().read_parquet(f"{GTFS_DATA_DICT.gcs_paths.DIGEST_GCS}raw/{GTFS_DATA_DICT.gtfs_digest_rollup.operator_summary}_{file_name}.parquet")

    # Prepare data for portfolio
    df2 = df[
        [
            "month_first_day",
            "analysis_name",
            "caltrans_district",
            "vp_name",
            "tu_name",
            "n_trips",
            "day_type",
            "daily_trips",
            "ttl_service_hours",
            "n_routes",
            "n_days",
            "n_shapes",
            "n_stops",
            "vp_messages_per_minute",
            "n_vp_trips",
            "daily_vp_trips",
            "pct_vp_trips",
            "n_vp_routes",
            "pct_vp_service_hours",
            "tu_messages_per_minute",
            "n_tu_trips",
            "daily_tu_trips",
            "pct_tu_trips",
            "n_tu_routes",
            "pct_tu_service_hours",
        ]
    ]
    df2.columns = df2.columns.str.replace("_", " ").str.title()

    df2 = df2.rename(
        columns={
            "Month First Day": "Date",
        }
    )
    df2.columns = df2.columns.str.replace("Vp", "VP").str.replace("Tu", "TU")

    # Create a couple of new columns
    df2["Percent of Trips with Trip Updates"] = (
        df2["N TU Trips"] / df2["N Trips"]
    ) * 100

    df2["Percent of Trips with Vehicle Positions"] = (
        df2["N VP Trips"] / df2["N Trips"]
    ) * 100

    """
    df2["Percent of Trips with Vehicle Positions"] = df2[
        "Percent of Trips with Vehicle Positions"
    ].clip(upper=100.0)
    df2["Percent of Trips with Trip Updates"] = df2[
        "Percent of Trips with Trip Updates"
    ].clip(upper=100.0)
    """
    # Save
    gcs_pandas().data_frame_to_parquet(df2,  f"{GTFS_DATA_DICT.gcs_paths.DIGEST_GCS}processed/{GTFS_DATA_DICT.gtfs_digest_rollup.operator_summary}_{file_name}.parquet")
    
    return df2

In [ ]:
monthly_operator_summary_clean = prep_operator_summary(
    file_name=file_name,
)

In [ ]:
monthly_operator_summary_clean.head(1)

In [ ]:
len(monthly_operator_summary_clean)

## 5) fct_monthly_routes 
* Add Caltrans District + Legislative District

In [ ]:
def load_fct_monthly_routes(
    project_name: str,
    date_col: str,
    dataset_name: str,
    start_date: str,
    end_date: str,
    file_name: str,
) -> pd.DataFrame:
    gdf = bq_utils.download_table(
        project_name=project_name,
        dataset_name=dataset_name,
        table_name=GTFS_DATA_DICT.gtfs_digest_rollup.route_map,
        date_col=date_col,
        start_date=start_date,
        end_date=end_date,
        geom_col="pt_array",
        geom_type="line",
    )

    # Merge with crosswalk
    crosswalk_df = load_crosswalk()[["name", "analysis_name", "caltrans_district"]]
    m1 = pd.merge(gdf, crosswalk_df, on=["name"], how="inner")

    utils.geoparquet_gcs_export(
        gdf=m1,
        gcs_file_path=f"{GTFS_DATA_DICT.gcs_paths.DIGEST_GCS}raw/",
        file_name=f"{GTFS_DATA_DICT.gtfs_digest_rollup.route_map}_{file_name}",
    )
    return m1

In [ ]:
monthly_routes_gdf = load_fct_monthly_routes(
    project_name=PROD_PROJECT,
    date_col=MONTH_DATE_COL,
    dataset_name=PROD_MART,
    start_date=previous_month,
    end_date=analysis_month,
    file_name=file_name,
)

In [ ]:

def prep_fct_monthly_routes(file_name: str) -> pd.DataFrame:
    """
    Prepare monthly route data by reading from GCS, cleaning, and exporting.
    """
    gdf = gpd.read_parquet(
        f"{GTFS_DATA_DICT.gcs_paths.DIGEST_GCS}raw/"
        f"{GTFS_DATA_DICT.gtfs_digest_rollup.route_map}_{file_name}.parquet",
        storage_options={"token": credentials.token},
    )

    # Keep only the most recent route geography
    gdf2 = (
        gdf.sort_values(
            by=["month_first_day", "analysis_name", "route_name"],
            ascending=[False, True, True],
        )
        .drop_duplicates(subset=["analysis_name", "route_name"])
    )

    # Drop unnecessary columns
    gdf2 = gdf2.drop(columns=["shape_id", "shape_array_key", "n_trips", "direction_id"])

    # Convert to miles
    gdf2["route_length_miles"] = (
        gdf2.geometry.to_crs(geography_utils.CA_NAD83Albers_ft).length / 5_280
    )

    # Clean column names
    gdf2.columns = gdf2.columns.str.replace("_", " ").str.title()

    # Export to GCS
    utils.geoparquet_gcs_export(
        gdf=gdf2,
        gcs_file_path=f"{GTFS_DATA_DICT.gcs_paths.DIGEST_GCS}processed/",
        file_name=f"{GTFS_DATA_DICT.gtfs_digest_rollup.route_map}_{file_name}",
    )

In [ ]:
monthly_routes_gdf = prep_fct_monthly_routes(
    project_name=PROD_PROJECT,
    date_col=MONTH_DATE_COL,
    dataset_name=PROD_MART,
    start_date=previous_month,
    end_date=analysis_month,
    file_name=file_name,
)

In [ ]:
monthly_routes_gdf["Caltrans District"].unique()

In [ ]:
monthly_routes_gdf.columns

In [ ]:
gdf = gpd.read_parquet(
    f"{GTFS_DATA_DICT.gcs_paths.DIGEST_GCS}processed/{GTFS_DATA_DICT.gtfs_digest_rollup.route_map}_{file_name}.parquet",
    storage_options={"token": credentials.token},
)

In [ ]:
gdf.columns

In [ ]:
# gdf.loc[gdf["Analysis Name"].str.contains("San Francisco")].drop(columns=["Geometry"])

In [ ]:
# gdf_og.loc[gdf_og.route_mile > 192][["name","route_name","geometry"]].explore()

## 5) fct_operator_hourly_summary

In [25]:
def load_fct_operator_hourly_summary(
    project_name: str,
    date_col: str,
    dataset_name: str,
    start_date: str,
    end_date: str,
    file_name: str,
) -> pd.DataFrame:
    df = bq_utils.download_table(
        project_name=project_name,
        dataset_name=dataset_name,
        table_name=GTFS_DATA_DICT.gtfs_digest_rollup.hourly_day_type_summary,
        date_col=date_col,
        start_date=start_date,
        end_date=end_date,
    )

    # Merge with crosswalk
    crosswalk_url = f"{GTFS_DATA_DICT.gcs_paths.DIGEST_GCS}processed/{GTFS_DATA_DICT.gtfs_digest_rollup.crosswalk}_{file_name}.parquet"

    crosswalk_df = gcs_pandas().read_parquet(crosswalk_url)[["name", "analysis_name",]]

    m1 = pd.merge(df, crosswalk_df, on="name", how="inner").drop_duplicates().reset_index()
    
    gcs_pandas().data_frame_to_parquet(m1, f"{GTFS_DATA_DICT.gcs_paths.DIGEST_GCS}raw/{GTFS_DATA_DICT.gtfs_digest_rollup.hourly_day_type_summary}_{file_name}.parquet")
   
    return m1

In [26]:
fct_operator_hourly_summary = load_fct_operator_hourly_summary(
    project_name=PROD_PROJECT,
    date_col=MONTH_DATE_COL,
    dataset_name=PROD_MART,
    start_date=last_year,
    end_date=analysis_month,
    file_name=file_name,
)

Downloading: 100%|██████████|
query: SELECT * FROM  `cal-itp-data-infra`.`mart_gtfs_rollup`.`fct_operator_hourly_summary` WHERE month_first_day >= DATE('2024-12-01') AND month_first_day <= DATE('2025-12-01')


In [27]:
fct_operator_hourly_summary.sample()

,index,month_first_day,name,day_type,departure_hour,n_trips,service_hours,flex_service_hours,analysis_name
3740,3740,2025-09-01,Bay Area 511 AC Transit Schedule,Saturday,1,20,16.42,0.00,Alameda-Contra Costa Transit District


In [28]:
fct_operator_hourly_summary.shape

(66752, 9)

In [ ]:
def prep_fct_operator_hourly_summary(file_name: str) -> pd.DataFrame:

    df = gcs_pandas().read_parquet(f"{GTFS_DATA_DICT.gcs_paths.DIGEST_GCS}raw/{GTFS_DATA_DICT.gtfs_digest_rollup.hourly_day_type_summary}_{file_name}.parquet")
    
    # Prepare data
    df2 = (
        df.groupby(["analysis_name", "month_first_day", "day_type", "departure_hour"])
        .agg({"n_trips": "sum"})
        .reset_index()
    )

    df2.columns = df2.columns.str.replace("_", " ").str.title()

    df2 = df2.rename(columns={"Month First Day": "Date"})

    df2["Date"] = df2["Date"].dt.strftime("%m-%Y")

    gcs_pandas().data_frame_to_parquet(df2, f"{GTFS_DATA_DICT.gcs_paths.DIGEST_GCS}processed/{GTFS_DATA_DICT.gtfs_digest_rollup.hourly_day_type_summary}_{file_name}.parquet")
    
    return df2

In [ ]:
clean_fct_operator_hourly_summary_df = prep_fct_operator_hourly_summary(
    file_name=file_name,
)

## Publish everything to GCS

### `gtfs_digest/publish_public_data.py`

In [ ]:
from typing import Literal

In [ ]:
GCS = GTFS_DATA_DICT["gtfs_digest_rollup"].dir

In [ ]:
GTFS_DATA_DICT["gtfs_digest_rollup"].

In [ ]:
df_file_keys = [
        "schedule_rt_route_direction",
        "operator_summary",
        "hourly_day_type_summary",
    ]

In [ ]:
gdf_file_keys = [
        "route_map"]

In [ ]:
def grab_filepaths(
    table_section: Literal["gtfs_digest_rollup"], 
    file_keys: list,
    file_name: str) -> list:
    """
    https://github.com/cal-itp/data-analyses/blob/main/_shared_utils/shared_utils/gtfs_analytics_data.yml
    
    table_section corresponds to "schedule_tables", "digest_tables", 
    "speeds_tables", etc
    """
    GCS = GTFS_DATA_DICT[table_section].dir
    
    file_paths = [GTFS_DATA_DICT[table_section][f] for f in file_keys]
    
    return [f"{GCS}processed/{f}_{file_name}.parquet" for f in file_paths]

In [ ]:
df_file_paths = grab_filepaths("gtfs_digest_rollup", df_file_keys, file_name)

In [ ]:
df_file_paths

In [ ]:
f"{GTFS_DATA_DICT.gcs_paths.DIGEST_GCS}processed/{GTFS_DATA_DICT.gtfs_digest_rollup.hourly_day_type_summary}_{file_name}.parquet"

In [ ]:
gdf_file_paths = grab_filepaths("gtfs_digest_rollup", gdf_file_keys, file_name)

In [ ]:
gdf_file_paths

In [ ]:
for f in df_file_paths + gdf_file_paths:
        publish_utils.write_to_public_gcs(
            f,
            f"gtfs_digest/{Path(f).name}",
            PUBLIC_GCS
        )

In [ ]:
def export_parquet_as_csv_or_geojson(
    filename: str,
    filetype: Literal["df", "gdf"],
):
    """
    For parquets, we want to export as csv.
    For geoparquets, we want to export as geojson.
    """
    if filetype=="df":
        df = gcs_pandas().read_parquet(filename)
        df.to_csv(
            f"{PUBLIC_GCS}gtfs_digest/"
            f"{Path(filename).stem}.csv", index=False
        )
        
        
    elif filetype=="gdf":
        df = gpd.read_parquet(filename, storage_options={"token": credentials.token},)
        utils.geojson_gcs_export(
            df,
            f"{PUBLIC_GCS}gtfs_digest/",
            Path(filename).stem,
            geojson_type = "geojson"
        )

In [ ]:
for f in df_file_paths:
   export_parquet_as_csv_or_geojson(f, filetype="df")

In [ ]:
for f in gdf_file_paths:
    publish_public_data.export_parquet_as_csv_or_geojson(f, filetype="gdf")